In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
%matplotlib inline

# Load dataset

In [2]:
#dataset = pd.read_csv('dataset/dev1_works_distance_dataset.csv')
dataset = pd.read_csv('dataset/pre1_works_distance_dataset.csv')
y = dataset['label'].values
features = dataset.columns.difference(['label'])
#features = ['title', 'primary_author', 'publisher.publisher_name']
print(features)
X = dataset[features].values
dataset.head()

Index(['work▪contributor▪contributorName', 'work▪dataSource▪sourceCode',
       'work▪dataSource▪sourcePrimaryKey',
       'work▪extent▪extentValue#3|work▪extent▪extentType',
       'work▪idno▪idno#CANINVALISBN13|work▪idno▪type',
       'work▪idno▪idno#CANINVALLCCN|work▪idno▪type',
       'work▪idno▪idno#CANISSN|work▪idno▪type',
       'work▪idno▪idno#DOI|work▪idno▪type',
       'work▪idno▪idno#INCORRISSN|work▪idno▪type',
       'work▪idno▪idno#OCLCNUM|work▪idno▪type',
       'work▪idno▪idno#PMID|work▪idno▪type',
       'work▪idno▪idno#VALISBN13|work▪idno▪type',
       'work▪idno▪idno#VALISSN|work▪idno▪type',
       'work▪idno▪idno#VALLCCN|work▪idno▪type', 'work▪itemAttributes▪issue',
       'work▪itemAttributes▪startPage', 'work▪itemAttributes▪volume',
       'work▪publicationCountry▪country',
       'work▪publisher▪name#8|work▪publisher▪role',
       'work▪relationship▪idno▪idno#HOSTITEM|work▪relationship▪type#VALISBN|work▪relationship▪idno▪type',
       'work▪relationship▪idno▪idno#

,work▪idno▪idno#PMID|work▪idno▪type,work▪idno▪idno#CANISSN|work▪idno▪type,work▪itemAttributes▪volume,work▪workDates▪startDate,work▪publisher▪name#8|work▪publisher▪role,work▪idno▪idno#VALISBN13|work▪idno▪type,work▪publicationCountry▪country,work▪idno▪idno#CANINVALISBN13|work▪idno▪type,work▪workTitle▪title#HASCOMMONTITLE|work▪workTitle▪type,work▪itemAttributes▪startPage,...,work▪workDates▪date,work▪idno▪idno#VALLCCN|work▪idno▪type,work▪relationship▪idno▪idno#HOSTITEM|work▪relationship▪type#VALISSN|work▪relationship▪idno▪type,label,work▪itemAttributes▪issue,work▪idno▪idno#VALISSN|work▪idno▪type,work▪dataSource▪sourcePrimaryKey,work▪dataSource▪sourceCode,work▪idno▪idno#CANINVALLCCN|work▪idno▪type,work▪relationship▪idno▪idno#HOSTITEM|work▪relationship▪type#VALISBN|work▪relationship▪idno▪type
0,0,0,0,0,31,0,100,0,100,0,...,100,0,0,0,0,0,0,0,0,0
1,0,0,0,0,31,0,100,0,100,0,...,100,0,0,0,0,0,0,0,0,0
2,0,0,0,100,100,0,100,0,100,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,100,100,0,0,0,100,0,...,100,0,0,0,0,0,0,0,0,0
4,0,0,0,100,36,0,0,0,100,0,...,0,0,0,0,0,0,0,0,0,0


## Split dataset by label (match|not_match) for visualisation

In [3]:
dataset_matched = dataset[dataset['label']==1]
dataset_not_matched = dataset[dataset['label']==0]

In [4]:
print(dataset_matched[features].mean())
print('__________________________')
print(dataset_not_matched[features].mean())

work▪contributor▪contributorName                                                                   72.388725
work▪dataSource▪sourceCode                                                                         39.996730
work▪dataSource▪sourcePrimaryKey                                                                   38.995176
work▪extent▪extentValue#3|work▪extent▪extentType                                                    3.020399
work▪idno▪idno#CANINVALISBN13|work▪idno▪type                                                        0.035238
work▪idno▪idno#CANINVALLCCN|work▪idno▪type                                                          0.138991
work▪idno▪idno#CANISSN|work▪idno▪type                                                               0.016352
work▪idno▪idno#DOI|work▪idno▪type                                                                   0.651541
work▪idno▪idno#INCORRISSN|work▪idno▪type                                                            0.414929
work▪idno▪idno#OCLC

In [5]:
print(sum(dataset_matched[features].mean()))
print(sum(dataset_not_matched[features].mean()))

595.714536833
536.891703676


In [6]:
def broken_plot(dataset, column, axis0_limits, axis1_limits, num_bins):
    f, axis = plt.subplots(2, 1, sharex=True)
    axis[0].hist(dataset[[column]].values, num_bins, alpha=0.5)
    axis[1].hist(dataset[[column]].values, num_bins, alpha=0.5)
    axis[0].set_ylim(axis0_limits[0], axis0_limits[1])
    axis[1].set_ylim(axis1_limits[0], axis1_limits[1])

    axis[0].spines['bottom'].set_visible(False)
    axis[1].spines['top'].set_visible(False)
    axis[0].xaxis.tick_top()
    axis[0].tick_params(labeltop='off')
    axis[1].xaxis.tick_bottom()
    d = .015
    kwargs = dict(transform=axis[0].transAxes, color='k', clip_on=False)
    axis[0].plot((-d,+d),(-d,+d), **kwargs)
    axis[0].plot((1-d,1+d),(-d,+d), **kwargs)
    kwargs.update(transform=axis[1].transAxes)
    axis[1].plot((-d,+d),(1-d,1+d), **kwargs)
    axis[1].plot((1-d,1+d),(1-d,1+d), **kwargs)

# Applying classification

## Splitting dataset to train and test

In [7]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [8]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [9]:
sss_cv = StratifiedShuffleSplit(y, 1, test_size=0.2, random_state=0)
for cv_index, test_index in sss_cv:
    cv_data, test_data = X[cv_index], X[test_index]
    cv_output, test_output = y[cv_index], y[test_index]
print(len([x for x in cv_output if x == 1]))
print(len(cv_output))
sss = StratifiedShuffleSplit(cv_output, 1, test_size=0.4, random_state=0)    
for train_index, val_index in sss:
    train_data, validation_data = cv_data[train_index], cv_data[val_index]
    train_output, validation_output = cv_output[train_index], cv_output[val_index]

19570
595101


In [10]:
model = linear_model.LogisticRegression(n_jobs=-1)
model.fit(train_data, train_output)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
model_features = pd.DataFrame(np.zeros((len(features), 2)), columns=['features', 'coeff'])
model_features['features'] = features
model_features['coeff'] = model.coef_[0]

In [12]:
model.score(validation_data, validation_output)

0.96897593271747307

In [13]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [14]:
#train_data_new
#for c in [0.001,0.005,0.01,0.05,0.1]:
#    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_data, train_output)
#    model1 = SelectFromModel(lsvc, prefit=True)
#    train_data_new = model1.transform(train_data)
#    lsvc_pred = lsvc.predict(validation_data)
#    fpr, tpr, thresholds = metrics.roc_curve(validation_output, lsvc_pred)
#    metrics.auc(fpr, tpr)

In [15]:
from sklearn import metrics
#predictions = model.predict(validation_data)
predictions = model.predict(validation_data)
fpr, tpr, thresholds = metrics.roc_curve(validation_output, predictions)
metrics.auc(fpr, tpr)

0.5748186968176664

In [19]:
train_predictions = model.predict(train_data)
fpr, tpr, thresholds = metrics.roc_curve(train_output, train_predictions)
metrics.auc(fpr, tpr)

0.5790315938834284

In [16]:
print(metrics.precision_score(validation_output, predictions))
print(metrics.recall_score(validation_output, predictions))
print(metrics.f1_score(validation_output, predictions))

0.613531522296
0.152912621359
0.244810307802


In [17]:
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for y, y_hat in zip(validation_output, predictions):
    if y == y_hat:
        if y_hat == 1:
            true_positive += 1
        else:
            true_negative += 1
    else:
        if y_hat == 1:
            false_positive += 1
        else:
            false_negative += 1
print('true_positive: %s' % true_positive)
print('true_negative: %s' % true_negative)
print('false_positive: %s' % false_positive)
print('false_negative: %s' % false_negative)

true_positive: 1197
true_negative: 229459
false_positive: 754
false_negative: 6631


# Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rfc = RandomForestClassifier(n_estimators=100)

In [23]:
rfc.fit(train_data, train_output)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [24]:
predictions = rfc.predict(validation_data)
fpr, tpr, thresholds = metrics.roc_curve(validation_output, predictions)
metrics.auc(fpr, tpr)

0.79536383160798185

In [26]:
predictions = rfc.predict(train_data)
fpr, tpr, thresholds = metrics.roc_curve(train_output, predictions)
metrics.auc(fpr, tpr)

0.84310424978286735

In [25]:
print(metrics.f1_score(validation_output, predictions))

0.682769726248
